# Comparing degenerations of spinor varieties

*Authors: George Balla, Daniel Corey, Igor Makhlin, and Victoria Schleis*

In this notebook accompanying our paper "Regular subdivisions, bounds on initial ideals, and categorical limits", we compute degenerations of spinor varieties. For spinor varieties of type $D_5$ and $D_6$, we compare $\mathrm{in}_w(I)$ and $I_w$ for tropical vectors $w$, where $I_w$ is the lower bound as defined in our paper. 

In [ ]:
using Oscar
using Combinatorics
pm = Polymake

include("InitialIdealsRegularSubdivisions.jl")
include("gfan_functions.jl");



## Spinor varieties


The space of isotropic subspaces of a $2n$-dimensional vector space equipped with a symmetric, nondegenerate form is an algebraic variety which consists of two irreducible components that are projectively equivalent to each other. The _spinor variety_ $\mathbb{S}_{n}$ is one of these irreducible components. It is a projective variety via the Wick embedding $J_n\subset S$ where $S$ is a polynomial ring with a generator for each subset of $[1,n]$ that has an even number of elements as $n$ modulo $2$. In [[Corey 2024](https://doi.org/10.1007/s40879-023-00715-z)], it is shown that $(J_n^{\circ})_w = \mathrm{in}_{w}(J_n)^{\circ}$ for $n\leq 5$ and $w\in \mathrm{Trop}(J_n)$. Here, we study the (stronger) equality $(J_n)_w = \mathrm{in}_{w}(J_n)$ for $n=5,6$ and $w\in \mathrm{Trop}(J_n)$.

---

**Function:** `spinor_variety_D(n)`

*Description*: For an integer `n` computes the ideal generating the spinor variety of type $D_n$.

In [ ]:
l(i, I) = length([j for j in I if j < i])

function spinor_variety_D(n)
    Le = vcat([reverse.(sort(reverse.(subsets(n, k)))) for k in 0:2:n]...)
    Lo = vcat([subsets(n, k) for k in 1:2:n]...)
    RDe, xDe = graded_polynomial_ring(QQ, :xDe => Le)
    xDe = Dict(k => v for (k, v) in zip(Le, xDe))

    relsDe = []
    for (I, J) in subsets(Lo, 2)
        SD = symdiff(I, J)
        if length(SD) >= 4
            push!(relsDe, sum([(-1)^(l(i, I) + l(i, J))*xDe[sort(symdiff(I, i))]*xDe[sort(symdiff(J, i))] for i in SD]))
        end
    end
    return ideal(relsDe)
end

*Example*: We compute ideals of the spinor varieties of type $D_5$ and $D_6$.

In [ ]:
I = spinor_variety_D(5)

J = spinor_variety_D(6);

## The case $n = 5 $

In this section we focus on the spinor variety associated to $D_5$, whose ideal `I` we computed in the previous section. 

We briefly recall the definitions of $I_w$ and $\Omega(I)$. Let $\Delta = \mathcal{A}(I)$ be the point configuration of $I$. A vector $w\in \mathbb{R}^{n}$ induces a regular subdivision $\mathrm{subd}_{w}(\Delta)$ of $\Delta$. We can form the ideals 

$$\{\tilde{I}_{B} \, : \, B \text{ is a maximal cell of } \mathrm{subd}_{w}(\Delta)\}$$

and the ideal they generate is

$$I_{w} = \sum_{B \text{ maximal}} \tilde{I}_{B}. $$

---

To show that $\mathrm{in}_w(I) = I_w$ for all tropical vectors $w$, we compute $\mathrm{Trop}(I)$ using `tropical_variety`. For each maximal cone in  $\mathrm{Trop}(I)$, we obtain a weight $w$ in the relative interior. We first verify that the maximal tropical is contained in the secondary cone of $\Delta$ containing $w$, to make sure that checking a single interior point is sufficient. We then compare the initial ideal $\mathrm{in}_w(I)$ (computed via `initial`) to the ideal $I_w$ we compute using `ideal_w` as documented in `Computing_I_w_and_Omega.ipynb`. We count the number of times equality is achieved and compare this value to the number of maximal cones of $\mathrm{Trop}(I)$, obtaining equality.


In [ ]:
R = base_ring(I)

RR, y = polynomial_ring(QQ, length(gens(R)), :y)
g = hom(R, RR, gens(RR))
II = g(I)
TV = tropical_variety(II, tropical_semiring_map(QQ, max))[1]
println(length(maximal_polyhedra(TV)))

D = point_configuration(I)

c = 0
for C in maximal_polyhedra(TV)
    rml = rays_modulo_lineality(C)[1]
    
    w = sum(rml)
    w = lcm(denominator.(w))*w
    
    SC = secondary_cone(subdivision_of_points(D, w))
    if !prod(r in SC for r in rml)
        println("!!!")
    end
    
    inI = initial(I, tropical_semiring_map(QQ), w)
    
    c += ideal_w(I, w) == inI
end
println(c)

## The case $n = 6$ 

We now turn to the spinor variety associated to $D_6$, given by the ideal `J` computed above. On the whole, we proceed very similarly to the case $n=5$, however, as the computation of the tropical variety takes a while, we load it from the result of a pre-computation using the function `parse_tropical_file` which we provide in the supplementary file `gfan_functions.jl`. 

For the sake of running time, this computation only performs the comparison for the last 1000 maximal cones in the tropical variety. It finds zero cases of equality, in contrast to the $n=5$ case.

In [ ]:
rs, M = parse_tropical_file("precomputed_data/n6trop.txt", true)

D = point_configuration(J)

c = 0
for i in nrows(M) - 1000 : nrows(M)
    rml = [-r for (j, r) in enumerate(rs) if M[i, j]]
    w = sum(rml)
    
    SC = secondary_cone(subdivision_of_points(D, w))
    if !prod(r in SC for r in rml)
        println("!!!")
    end    
    
    inJ = initial(J, tropical_semiring_map(QQ), w)
    Jw = ideal_w(J, w)
    
    c += Jw == inJ
end
c